# EDA/ sanity check 
Check on the constructed enron network to determine if the graph is a good use case

going to check two CEO/president who have a very different embedding in graphcase and investigate the underlying reason.

In [1]:
# run this only once!!
import os
os. chdir("../..")
os.getcwd()

'/Users/tonpoppe/workspace/graphcase_experiments/graphcase_experiments'

In [2]:
import networkx as nx
from graphcase_experiments.algos.GraphCaseWrapper import GraphCaseWrapper
from graphcase_experiments.tools.embedding_plotter import plot_embedding2, plotly_embedding, plot_graph
from graphcase_experiments.tools.calculate_embed import calculate_graphcase_embedding
import matplotlib.pyplot as plt
import pydot
import contextlib

/Users/tonpoppe/opt/anaconda3/envs/graphcase/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load graph
SOURCE_PATH = 'graphcase_experiments/graphs/enron/data/'
G = nx.read_gpickle(SOURCE_PATH + 'enron_sub_graph.pickle')

In [4]:
algo = GraphCaseWrapper
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        _, tbl_gc = calculate_graphcase_embedding(
            G, algo.LOCATION, params=algo.COMP_PARAMS, verbose=False, algo=algo
        )
plotly_embedding(tbl_gc)

2022-07-15 10:11:18.493673: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-15 10:12:38.136997: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/Users/tonpoppe/workspace/graphcase_experiments/graphcase_experiments/graphcase_experiments/tools/embedding_plotter.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  embed_df['embed1'] = embed[:,0]
/Users/tonpoppe/workspace/graphcas

# Comparision of network with article of Creamer

First checking the # of emails

In [15]:
from graphcase_experiments.graphs.enron.mail_reader import spark, Enron_to_graph
import pyspark.sql.functions as F
tmp_path = '/Users/tonpoppe/Downloads/'
nodes = (spark.read.format('parquet').load(tmp_path + 'nodes')
    .filter("isCore= 1")
)
tmp_path = '/Users/tonpoppe/Downloads/'
edges = spark.read.format('parquet').load(tmp_path + 'edges')

In [21]:
core_edges = (edges
    .join(nodes.select(F.col('email_address').alias('source')), 'source', 'inner')
    .join(nodes.select(F.col('email_address').alias('target')), 'target', 'inner')
)
core_edges.show(4)

+---------+---------+------+------+-------+------+
|   target|   source|weight|cnt_cc|size_to|  size|
+---------+---------+------+------+-------+------+
|stclair-c|  panus-s|   104|     2|  57712| 58100|
|kitchen-l| taylor-m|    67|    19|  95838|129628|
| watson-k|  lokay-m|    76|     7|  54624| 60276|
|   neal-s|kitchen-l|    15|     1|  17083| 19712|
+---------+---------+------+------+-------+------+
only showing top 4 rows



In [28]:
core_edges_inc = (core_edges
    .groupBy('target').agg(F.sum('weight').alias('total_to'),
                        F.sum('cnt_cc').alias('total_cc'))
    .withColumnRenamed('target', 'email')
)
core_edges_out = (core_edges
    .groupBy('source').agg(F.sum('weight').alias('total_from'))
    .withColumnRenamed('source', 'email')
)
core_egdes_stat = (core_edges_inc.join(core_edges_out, 'email', 'outer')
    .fillna(0)
    .withColumn('total', F.col('total_to') + F.col('total_cc') + F.col('total_from'))
)
core_egdes_stat.show(10)

+----------+--------+--------+----------+-----+
|     email|total_to|total_cc|total_from|total|
+----------+--------+--------+----------+-----+
|   allen-p|     310|      33|       320|  663|
|  arnold-j|     388|      26|       427|  841|
|   arora-h|      71|       4|        16|   91|
|  badeer-r|     252|      28|        16|  296|
|  bailey-s|     894|     181|       316| 1391|
|    bass-e|     221|      59|       391|  671|
|baughman-d|      36|       1|       151|  188|
|    beck-s|     156|      70|       340|  566|
|  benson-r|      40|       6|         0|   46|
|   blair-l|     108|      38|       213|  359|
+----------+--------+--------+----------+-----+
only showing top 10 rows



In [29]:
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
assembler = VectorAssembler().setInputCols(['total']).setOutputCol("features")
transformed = assembler.transform(core_egdes_stat)
scaler = MinMaxScaler(inputCol="features", outputCol="scaledtotal")
scalerModel =  scaler.fit(transformed.select("features"))
scaledData = scalerModel.transform(transformed)
scaledData.show(10)

+----------+--------+--------+----------+-----+--------+--------------------+
|     email|total_to|total_cc|total_from|total|features|         scaledtotal|
+----------+--------+--------+----------+-----+--------+--------------------+
|   allen-p|     310|      33|       320|  663| [663.0]|[0.18960068945705...|
|  arnold-j|     388|      26|       427|  841| [841.0]|[0.24073542085607...|
|   arora-h|      71|       4|        16|   91|  [91.0]|[0.02528009192760...|
|  badeer-r|     252|      28|        16|  296| [296.0]|[0.08417121516805...|
|  bailey-s|     894|     181|       316| 1391|[1391.0]|[0.39873599540361...|
|    bass-e|     221|      59|       391|  671| [671.0]|[0.19189887963228...|
|baughman-d|      36|       1|       151|  188| [188.0]|[0.05314564780235...|
|    beck-s|     156|      70|       340|  566| [566.0]|[0.16173513358230...|
|  benson-r|      40|       6|         0|   46|  [46.0]|[0.01235277219189...|
|   blair-l|     108|      38|       213|  359| [359.0]|[0.10226

Over het algemeem klopt het wel, maar lyn blair heeft in onze graph 50% lagere count.

In [30]:
#check the input.
tmp_path = '/Users/tonpoppe/Downloads/'
email = spark.read.format('parquet').load(tmp_path + 'enron_email')

In [42]:
res = email.filter("recipient like '%blair%'").groupBy('from_address').count().orderBy('count', ascending=False)
res.show(5)

+--------------------+-----+
|        from_address|count|
+--------------------+-----+
|             blair-l|  306|
|john.buchanan@enr...|  122|
|lorna.brennan@enr...|  106|
|janet.butler@enro...|   74|
|            corman-s|   73|
+--------------------+-----+
only showing top 5 rows



In [54]:
own = email.filter("recipient like '%blair%' and from_address = 'blair-l'")
own.count()

306

In [50]:
path = "/Users/tonpoppe/Downloads/enron_parsed_all3_dedup"  #all
df = spark.read.format('parquet').load(path)
tmp = df.join(own.select('mail_id'), 'mail_id', 'inner')
tmp.select('folder', 'fname').show(5, truncate=False)
# visual inspectation of these email show indeed that some people send to them self.

+--------------------------------+----------------------------------------------------------------------+
|folder                          |fname                                                                 |
+--------------------------------+----------------------------------------------------------------------+
|blair-l/sent_items              |/Users/tonpoppe/Downloads/maildir3/blair-l/sent_items/78.             |
|blair-l/sent_items              |/Users/tonpoppe/Downloads/maildir3/blair-l/sent_items/723.            |
|blair-l/deleted_items/_sent_mail|/Users/tonpoppe/Downloads/maildir3/blair-l/deleted_items/_sent_mail/6.|
|blair-l/personnel___paa         |/Users/tonpoppe/Downloads/maildir3/blair-l/personnel___paa/16.        |
|blair-l/sent_items              |/Users/tonpoppe/Downloads/maildir3/blair-l/sent_items/127.            |
+--------------------------------+----------------------------------------------------------------------+
only showing top 5 rows



In [53]:
tmp =core_edges.filter("source = 'blair-l'").groupBy('target').agg(F.sum('weight').alias('tot_cnt'))
tmp.orderBy('tot_cnt', ascending=False).show(10)

+-------------+-------+
|       target|tot_cnt|
+-------------+-------+
|     corman-s|    119|
|     watson-k|     26|
|schoolcraft-d|     24|
|      lokay-m|     10|
|     donoho-l|      9|
|     ybarbo-p|      5|
|      lokey-t|      4|
|    mccarty-d|      4|
|     fossum-d|      4|
|       rapp-b|      3|
+-------------+-------+
only showing top 10 rows



# time span 
Check time span of the e-mail

In [57]:
# no data are stored in the enron parsing file. need to reparce with data. -> on to do list

# content type
Check content types

In [59]:
email.groupBy('content_type').count().show(5)

+------------+-------+
|content_type|  count|
+------------+-------+
|  text/plain|1384662|
+------------+-------+



In [61]:
email.show()

+--------------------+--------------------+----------+--------------------+------+--------------------+------+--------------------+-------+------------+--------------------+-----+-----+------+
|           recipient|        from_address|email_size|          to_address|cnt_to|          cc_address|cnt_cc|         bcc_address|cnt_bcc|content_type|             mail_id|is_to|is_cc|is_bcc|
+--------------------+--------------------+----------+--------------------+------+--------------------+------+--------------------+-------+------------+--------------------+-----+-----+------+
|'.'annette@enron.com|           lenhart-m|       762|[chad.landry@enro...|    19|                  []|     0|                  []|      0|  text/plain|F05O2001074739070...|    1|    0|     0|
|'.'lcentol@enron.com|           lenhart-m|       762|[chad.landry@enro...|    19|                  []|     0|                  []|      0|  text/plain|F05O2001074739070...|    1|    0|     0|
|   '.'pete@enron.com|           ge

22/07/16 01:13:43 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 998437 ms exceeds timeout 120000 ms
22/07/16 01:13:43 WARN SparkContext: Killing executors is not supported by current scheduler.
22/07/24 21:35:30 WARN TransportChannelHandler: Exception in connection from /10.0.0.9:54814
java.io.IOException: Operation timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:378)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:253)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:350)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.